In [36]:
from PIL import Image
from rembg import new_session, remove
import pandas as pd
import numpy as np
from dataclasses import dataclass
import pandas as pd

Identifier = str
Link = str



ModuleNotFoundError: No module named 'PIL'

In [39]:
class Selector:
    _DFoutfit: pd.DataFrame
    _DFproduct: pd.DataFrame
    
    def __init__(self) -> None:
        self._DFoutfit = pd.read_csv('../dataset/outfit_data.csv')
        self._DFproduct = pd.read_csv('../dataset/product_data.csv')
    
    @staticmethod
    def get_pixel_matrix(image):
        pixel_data = list(image.getdata())
        width, height = image.size
        pixel_matrix = [pixel_data[i * width:(i + 1) * width] for i in range(height)]
        return pixel_matrix

    def get_all_outfits(self) -> list[str]:
        return self.DF_outfits()['cod_outfit'].unique()

    def outfits_with(self, prod_id: list[Identifier]) -> list[str]:
        return self.DF_outfits() [self.DF_outfits() ['cod_modelo_color'] == prod_id]['cod_outfit'].to_list()
    
    def outfits_check(self, prod_id: list[Identifier]) -> bool:
        return self.DF_outfits() [self.DF_outfits() ['cod_modelo_color'] == prod_id]['cod_outfit'].nunique() == 1
    
    def get_outfit(self, outf_id: Identifier) -> list[Identifier]:
        return self.DF_outfits() [self.DF_outfits() ['cod_outfit'] == outf_id]['cod_modelo_color'].to_list()
    
    def get_product_info(self, prod_id: Identifier) -> dict:
        return self.DF_products().loc[prod_id].to_dict()
    
    def get_product_image(self, prod_id: Identifier) -> dict:
        filename: str = self.get_product_info(prod_id)['des_filename']
        return "../dataset/images/"+filename.split("/")[-1]
    
    def show_product(self, prod_id: Identifier) -> None:
        jpg_path: str = self.get_product_info(prod_id)['des_filename']
        img = Image.open(jpg_path)
        img.show()
        return
    
    def show_outfit(self, outf_id: Identifier) -> None:
        products: list[Identifier] = self.get_outfit(outf_id)
        jpg_paths: list[str] = [self.get_product_image(prod_id) for prod_id in products]
        images: list = [Image.open(path) for path in jpg_paths]

        # Determine the size of the collage
        total_width = sum(img.width for img in images)
        max_height = max(img.height for img in images)

        # Create a new image with white background
        collage = Image.new('RGB', (total_width, max_height), (255, 255, 255))

        # Paste each image into the collage
        current_width = 0
        for img in images:
            collage.paste(img, (current_width, 0))
            current_width += img.width

        # Save the collage
        collage.save("../dataset/outfits/"+outf_id)

    def get_rgb(self, id):
        input_path = self.get_product_image(id)
        n = len(input_path)
        no_back_path = input_path[:n-4] + "_nback.png"
        img = None
        try:  
            img = Image.open(no_back_path)
        except FileNotFoundError:
            in_file = Image.open(input_path)
            output = remove(in_file)
            output.save(no_back_path)
            img = output
        
        pixel_matrix = get_pixel_matrix(img)
        width,height = img.size
        suma = np.array([0,0,0,0], dtype=float)
        print(pixel_matrix[0][0])
        cnt = 0
        for i in range(height):
            for j in range(width):
                col = np.array(pixel_matrix[i][j])
                if np.all(abs(col) != np.array([0,0,0,0])):
                    suma = suma + col
                    cnt += 1
        print(suma/cnt)
        return suma/cnt

    # GETTERS
    
    def DF_outfits(self) -> pd.DataFrame:
        return self._DFoutfit
        
    def DF_products(self) -> pd.DataFrame:
        return self._DFproduct


In [65]:
selector = Selector()
selector.DF_products()['des_color_specification_esp'].unique()


array(['OFFWHITE', 'TEJANO OSCURO', 'ROSA PASTEL', 'MOSTAZA', 'ROJO',
       'GRIS MEDIO VIGORE', 'CARAMELO', 'KHAKI', 'GRIS OSCURO VIGORE',
       'CRUDO', 'FRESA', 'BEIGE', 'CHOCOLATE', 'NEGRO', 'ROSA PALO',
       'TEJANO MEDIO', 'AMARILLO', 'BLANCO', 'ARENA', 'MORADO', 'MARRON',
       'AZUL', 'AGUA', 'NAVY', 'TEJANO GRIS OSCURO', 'CAMEL', 'VERDE',
       'ROSA', 'FUCSIA', 'GRIS CLARO VIGORE', 'CUERO', 'VISON',
       'TEJANO CLARO', 'PLATA', 'GRIS', 'NUDE', 'ORO', 'TURQUESA',
       'TEJANO NEGRO', 'NARANJA', 'DIRTY', 'HIELO', 'CELESTE', 'BURDEOS',
       'AMARILLO FLUOR', 'PIEDRA', 'MALVA', 'ROSA LIGHT', 'MARINO',
       'MANDARINA', 'TERRACOTA', 'CALDERO', 'GRANATE', 'TOPO',
       'VERDE PASTEL', 'AZUL NOCHE', 'MISTERIO', 'LIMA', 'COFFEE',
       'PERLA', 'ESMERALDA', 'OCRE', 'VIOLETA', 'VINO', 'MARFIL',
       'BOTELLA', 'ANTRACITA', 'PEACH', 'VAINILLA', 'TABACO', 'ELECTRICO',
       'AMARILLO PASTEL', 'PORCELANA', 'TAUPE', 'COBRE', 'CANELA',
       'PETROLEO', 'CORAL', 'LILA'

In [67]:
color_mapping = {
    'OFFWHITE': (255, 255, 255),
    'TEJANO OSCURO': (25, 25, 112),
    'ROSA PASTEL': (255, 182, 193),
    'MOSTAZA': (255, 255, 0),
    'ROJO': (255, 0, 0),
    'GRIS MEDIO VIGORE': (128, 128, 128),
    'CARAMELO': (210, 105, 30),
    'KHAKI': (240, 230, 140),
    'GRIS OSCURO VIGORE': (40, 40, 40),
    'CRUDO': (255, 235, 205),
    'FRESA': (255, 0, 128),
    'BEIGE': (245, 245, 220),
    'CHOCOLATE': (139, 69, 19),
    'NEGRO': (0, 0, 0),
    'ROSA PALO': (255, 192, 203),
    'TEJANO MEDIO': (70, 130, 180),
    'AMARILLO': (255, 255, 0),
    'BLANCO': (255, 255, 255),
    'ARENA': (194, 178, 128),
    'MORADO': (128, 0, 128),
    'MARRON': (165, 42, 42),
    'AZUL': (0, 0, 255),
    'AGUA': (0, 255, 255),
    'NAVY': (0, 0, 128),
    'TEJANO GRIS OSCURO': (169, 169, 169),
    'CAMEL': (193, 154, 107),
    'VERDE': (0, 128, 0),
    'ROSA': (255, 0, 255),
    'FUCSIA': (255, 0, 255),
    'GRIS CLARO VIGORE': (192, 192, 192),
    'CUERO': (139, 69, 19),
    'VISON': (184, 134, 11),
    'TEJANO CLARO': (135, 206, 250),
    'PLATA': (192, 192, 192),
    'GRIS': (128, 128, 128),
    'NUDE': (255, 222, 173),
    'ORO': (255, 215, 0),
    'TURQUESA': (64, 224, 208),
    'TEJANO NEGRO': (0, 0, 0),
    'NARANJA': (255, 165, 0),
    'DIRTY': (30, 144, 255),
    'HIELO': (176, 224, 230),
    'CELESTE': (173, 216, 230),
    'BURDEOS': (128, 0, 32),
    'AMARILLO FLUOR': (255, 255, 0),
    'PIEDRA': (105, 105, 105),
    'MALVA': (221, 160, 221),
    'ROSA LIGHT': (255, 182, 193),
    'MARINO': (0, 0, 128),
    'MANDARINA': (255, 165, 0),
    'TERRACOTA': (204, 78, 92),
    'CALDERO': (139, 69, 19),
    'GRANATE': (128, 0, 0),
    'TOPO': (135, 125, 115),
    'VERDE PASTEL': (152, 251, 152),
    'AZUL NOCHE': (25, 25, 112),
    'MISTERIO': (112, 138, 144),
    'LIMA': (0, 255, 0),
    'COFFEE': (165, 42, 42),
    'PERLA': (240, 240, 240),
    'ESMERALDA': (0, 201, 87),
    'OCRE': (204, 119, 34),
    'VIOLETA': (238, 130, 238),
    'VINO': (128, 0, 0),
    'MARFIL': (255, 255, 240),
    'BOTELLA': (0, 128, 0),
    'ANTRACITA': (51, 51, 51),
    'PEACH': (255, 218, 185),
    'VAINILLA': (245, 222, 179),
    'TABACO': (128, 78, 40),
    'ELECTRICO': (125, 249, 255),
    'AMARILLO PASTEL': (255, 255, 153),
    'PORCELANA': (240, 240, 240),
    'TAUPE': (72, 60, 50),
    'COBRE': (184, 115, 51),
    'CANELA': (210, 105, 30),
    'PETROLEO': (0, 128, 128),
    'CORAL': (255, 127, 80),
    'LILA': (200, 162, 200),
    'COGNAC': (143, 89, 34),
    'OLIVA': (128, 128, 0),
    'MENTA': (152, 255, 152),
    'TEJANO GRIS CLARO': (192, 192, 192),
    'TEJANO GRIS': (128, 128, 128),
    'DIRTY OSCURO': (34, 49, 63),
    'BLEACH': (255, 252, 220),
    'NARANJA PASTEL': (255, 223, 186),
    'CAZA': (139, 69, 19),
    'BLOOD': (102, 0, 0),
    'CEREZA': (222, 49, 99),
    'CENIZA': (192, 192, 192),
    'CURRY': (205, 185, 125),
    'BLUEBLACK': (0, 0, 139),
    'SALMON': (250, 128, 114),
    'CHICLE': (255, 182, 193),
    'TEJANO SOFT': (112, 128, 144),
    'GUNMETAL': (42, 52, 57),
    'MANZANA': (50, 205, 50),
    'BILLAR': (0, 100, 0),
    'MUSGO': (173, 223, 173),
    'TINTA': (0, 0, 0),
    'INDIGO': (75, 0, 130),
    'ROSA FLUOR': (255, 20, 147),
    'DIRTY CLARO': (220, 220, 220),
    'CIRUELA': (139, 0, 139),
    'BERMELLON': (227, 66, 52),
    'GERANIO': (201, 0, 22),
    'PIMENTON': (153, 0, 0),
    'PRUSIA': (0, 49, 83),
    'ASFALTO': (47, 79, 79),
}

Could not find RGB value for OFFWHITE
Could not find RGB value for TEJANO OSCURO
Could not find RGB value for ROSA PASTEL
Could not find RGB value for MOSTAZA
Could not find RGB value for ROJO
Could not find RGB value for GRIS MEDIO VIGORE
Could not find RGB value for CARAMELO
Could not find RGB value for GRIS OSCURO VIGORE
Could not find RGB value for CRUDO
Could not find RGB value for FRESA
Could not find RGB value for NEGRO
Could not find RGB value for ROSA PALO
Could not find RGB value for TEJANO MEDIO
Could not find RGB value for AMARILLO
Could not find RGB value for BLANCO
Could not find RGB value for ARENA
Could not find RGB value for MORADO
Could not find RGB value for MARRON
Could not find RGB value for AZUL
Could not find RGB value for AGUA
Could not find RGB value for TEJANO GRIS OSCURO
Could not find RGB value for CAMEL
Could not find RGB value for VERDE
Could not find RGB value for ROSA
Could not find RGB value for FUCSIA
Could not find RGB value for GRIS CLARO VIGORE
Coul